**1.0 Introdução**

Esse codigo tem a finalidade de preparar os dados do cartola para serem input do modelo de Machine Learning. 

Há o carregamento do dado, limpeza da base, criação de variaveis de estatistica e salvamento da base tratada.

**2.0 Pacotes**
 
Importar pacotes.

In [33]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

**3.0 Dados**

Carregar dados necessários, já salvos.

**3.1 Partidas**

Realizar tratamento de dados das partidas.

In [34]:
#carregar dados de partidas
partidas = (pd
.read_csv("C:/Users/eduar/OneDrive/Área de Trabalho/Python/cartola_fc/bronze/bases/cartola_partidas.csv")
)

#carregar dados de clubes
clubes = (pd
.read_csv("C:/Users/eduar/OneDrive/Área de Trabalho/Python/cartola_fc/bronze/bases/cartola_clubes.csv")
.drop(['abreviacao','slug', 'apelido', 'nome', 'url_editoria',
       'escudos.60x60', 'escudos.45x45', 'escudos.30x30'], axis=1)
)

# Realizar o join e correção dos clubes visitantes
partidas = pd.merge(partidas, clubes, left_on='clube_visitante_id', right_on='id', how='left')
partidas.drop(['id'], axis=1, inplace=True)  # Use inplace=True para evitar reatribuições desnecessárias
partidas.rename(columns={'nome_fantasia': 'clube_visitante'}, inplace=True)  # Renomear a coluna

# Realizar o join e correção dos clubes mandantes
partidas = pd.merge(partidas, clubes, left_on='clube_casa_id', right_on='id', how='left')
partidas.drop(['id'], axis=1, inplace=True)  # Use inplace=True para evitar reatribuições desnecessárias
partidas.rename(columns={'nome_fantasia': 'clube_casa',
                         'local': 'estadio',
                         'placar_oficial_visitante': 'gols_visitante',
                         'placar_oficial_mandante': 'gols_casa',
                         'clube_visitante_posicao': 'visitante_classificacao',
                         'clube_casa_posicao': 'casa_classificacao',
                        }, inplace=True)

#select colunas
partidas = partidas[['estadio','gols_visitante',
                     'gols_casa','visitante_classificacao',
                     'clube_visitante','casa_classificacao','clube_casa','rodada']
                     ]

#visualizar dados
display(partidas.head(5))

,estadio,gols_visitante,gols_casa,visitante_classificacao,clube_visitante,casa_classificacao,clube_casa,rodada
0,,NaN,NaN,0,Grêmio,0,Fluminense,38
1,,NaN,NaN,0,Bragantino,0,Vasco,38
2,,NaN,NaN,0,Flamengo,0,São Paulo,38
3,,NaN,NaN,0,Fortaleza,0,Santos,38
4,,NaN,NaN,0,América-MG,0,Goiás,38


In [35]:
#partidas mandantes
partidas_casa = partidas.copy()

#renomear colunas
partidas_casa.rename(columns={'clube_casa': 'equipe',
                         'clube_visitante': 'adv',
                         'gols_visitante': 'gols_sofridos',
                         'gols_casa': 'gols_marcados',
                         'visitante_classificacao': 'classificacao_adv',
                         'casa_classificacao': 'classificacao',
                        }, inplace=True)

#criar coluna mandante
partidas_casa['mandante'] = 1

#organizar colunas
partidas_casa = partidas_casa[['rodada','equipe','adv','gols_marcados','gols_sofridos','mandante','estadio','classificacao','classificacao_adv'
]]

display(partidas_casa.head(5))

,rodada,equipe,adv,gols_marcados,gols_sofridos,mandante,estadio,classificacao,classificacao_adv
0,38,Fluminense,Grêmio,NaN,NaN,1,,0,0
1,38,Vasco,Bragantino,NaN,NaN,1,,0,0
2,38,São Paulo,Flamengo,NaN,NaN,1,,0,0
3,38,Santos,Fortaleza,NaN,NaN,1,,0,0
4,38,Goiás,América-MG,NaN,NaN,1,,0,0


In [36]:
#partidas visitante
partidas_visitante = partidas.copy()

#renomear colunas
partidas_visitante.rename(columns={'clube_casa': 'adv',
                         'clube_visitante': 'equipe',
                         'gols_visitante': 'gols_marcados',
                         'gols_casa': 'gols_sofridos',
                         'visitante_classificacao': 'classificacao',
                         'casa_classificacao': 'classificacao_adv',
                        }, inplace=True)

#criar coluna mandante
partidas_visitante['mandante'] = 0

#organizar colunas
partidas_visitante = partidas_visitante[['rodada','equipe','adv','gols_marcados',
                                         'gols_sofridos','mandante','estadio','classificacao','classificacao_adv']]


display(partidas_visitante.head(5))

,rodada,equipe,adv,gols_marcados,gols_sofridos,mandante,estadio,classificacao,classificacao_adv
0,38,Grêmio,Fluminense,NaN,NaN,0,,0,0
1,38,Bragantino,Vasco,NaN,NaN,0,,0,0
2,38,Flamengo,São Paulo,NaN,NaN,0,,0,0
3,38,Fortaleza,Santos,NaN,NaN,0,,0,0
4,38,América-MG,Goiás,NaN,NaN,0,,0,0


In [37]:
#unificar
partidas_geral = partidas_visitante.append(partidas_casa)


# Defina a função para calcular o resultado
def DefResultado(row):
    if row['gols_marcados'] > row['gols_sofridos']:
        return 1
    elif row['gols_marcados'] < row['gols_sofridos']:
        return 0
    else:
        return 0.5
    
# Defina a função para calcular o resultado
def SaldoGols(row):
    if row['gols_sofridos'] == 0:
        return 1
    else:
        return 0
    
# Defina a função para calcular o resultado
def SaldoGolsAtq(row):
    if row['gols_marcados'] == 0:
        return 1
    else:
        return 0
# Aplique a função
partidas_geral['pontos_time'] = partidas_geral.apply(DefResultado, axis=1)

# Aplique a função
partidas_geral['SG_def'] = partidas_geral.apply(SaldoGols, axis=1)

# Aplique a função
partidas_geral['SG_Atq'] = partidas_geral.apply(SaldoGolsAtq, axis=1)

#Saldo de gols
partidas_geral['saldo_gols'] = partidas_geral['gols_marcados'] - partidas_geral['gols_sofridos']

#oranizar colunas
partidas_geral = partidas_geral[['rodada',
                                         'equipe',
                                         'adv',
                                         'pontos_time',
                                         'gols_marcados',
                                         'gols_sofridos',
                                         'saldo_gols',
                                         'SG_def',
                                         'SG_Atq',
                                         'mandante',
                                         'estadio',
                                         'classificacao',
                                         'classificacao_adv']
                                         ]

display(partidas_geral.head(5))

,rodada,equipe,adv,pontos_time,gols_marcados,gols_sofridos,saldo_gols,SG_def,SG_Atq,mandante,estadio,classificacao,classificacao_adv
0,38,Grêmio,Fluminense,0.5,NaN,NaN,NaN,0,0,0,,0,0
1,38,Bragantino,Vasco,0.5,NaN,NaN,NaN,0,0,0,,0,0
2,38,Flamengo,São Paulo,0.5,NaN,NaN,NaN,0,0,0,,0,0
3,38,Fortaleza,Santos,0.5,NaN,NaN,NaN,0,0,0,,0,0
4,38,América-MG,Goiás,0.5,NaN,NaN,NaN,0,0,0,,0,0


**3.1.1 Estatisticas PRÓ**

Tratamento de dados das partidas, para estatísticas.

In [38]:
#cria funcao para tratamento
def CalculaEstatisticas(df, coluna, id,medida):

    #media geral
    df[medida+'Avg'+coluna] =  df.groupby(id)[coluna].transform(lambda x: x.shift().rolling(40, min_periods=1).mean()).round(2)

    #ultimo
    df[medida+'Last'+coluna] = df.groupby(id)[coluna].transform(lambda x: x.shift().rolling(1, min_periods=1).mean()).round(2)
    
    #media movel5
    df[medida+'AvgMov5'+coluna] = df.groupby(id)[coluna].transform(lambda x: x.shift().rolling(5, min_periods=1).mean()).round(2)

    #media movel10
    df[medida+'AvgMov10'+coluna] = df.groupby(id)[coluna].transform(lambda x: x.shift().rolling(10, min_periods=1).mean()).round(2)

    # Substitui NaN nas colunas criadas pelo valor original da coluna 'coluna'
    for col in [medida+'Avg'+coluna, medida+'Last'+coluna, medida+'AvgMov5'+coluna, medida+'AvgMov10'+coluna]:
        df[col].fillna(df[coluna], inplace=True)

In [39]:
#Gerar Estatísticas dos confrontos
colunas_numeros = ['pontos_time','gols_marcados','gols_sofridos',
                    'saldo_gols','classificacao','SG_Atq',
                    'classificacao_adv','SG_def']

#colunas para remover
remove_columns = ['pontos_time','gols_marcados','gols_sofridos',
                    'saldo_gols','classificacao','SG_def','SG_Atq',
                    'classificacao_adv','adv']

#estatistica
partidas_pro = partidas_geral.copy()

# Ordenar o DataFrame por pessoa e nprova
partidas_pro = partidas_pro.sort_values(by=['equipe', 'rodada'])

#for para criar estatisticas
for coluna in colunas_numeros:

    #criar estatiscas para a coluna
    CalculaEstatisticas(partidas_pro, coluna,"equipe","partidas_pro")
    print("Estatística criada para coluna",coluna)

#substituir com zeros e remover scouts da rodada
partidas_pro = partidas_pro.drop(remove_columns, axis=1)

#ver
display(partidas_pro.head(5))

Estatística criada para coluna pontos_time
Estatística criada para coluna gols_marcados
Estatística criada para coluna gols_sofridos
Estatística criada para coluna saldo_gols
Estatística criada para coluna classificacao


Estatística criada para coluna SG_Atq
Estatística criada para coluna classificacao_adv
Estatística criada para coluna SG_def


,rodada,equipe,mandante,estadio,partidas_proAvgpontos_time,partidas_proLastpontos_time,partidas_proAvgMov5pontos_time,partidas_proAvgMov10pontos_time,partidas_proAvggols_marcados,partidas_proLastgols_marcados,...,partidas_proAvgMov5SG_Atq,partidas_proAvgMov10SG_Atq,partidas_proAvgclassificacao_adv,partidas_proLastclassificacao_adv,partidas_proAvgMov5classificacao_adv,partidas_proAvgMov10classificacao_adv,partidas_proAvgSG_def,partidas_proLastSG_def,partidas_proAvgMov5SG_def,partidas_proAvgMov10SG_def
371,1,América-MG,1,Independência,0.0,0.0,0.0,0.0,0.00,0.0,...,1.00,1.00,8.0,8.0,8.0,8.0,0.0,0.0,0.0,0.0
361,2,América-MG,0,Morumbi,0.0,0.0,0.0,0.0,0.00,0.0,...,1.00,1.00,8.0,8.0,8.0,8.0,0.0,0.0,0.0,0.0
353,3,América-MG,0,Vila Belmiro,0.0,0.0,0.0,0.0,0.00,0.0,...,1.00,1.00,8.5,9.0,8.5,8.5,0.0,0.0,0.0,0.0
342,4,América-MG,1,Independência,0.0,0.0,0.0,0.0,0.67,2.0,...,0.67,0.67,10.0,13.0,10.0,10.0,0.0,0.0,0.0,0.0
331,5,América-MG,0,Nabi Abi Chedid,0.0,0.0,0.0,0.0,0.75,1.0,...,0.50,0.50,10.0,10.0,10.0,10.0,0.0,0.0,0.0,0.0


In [40]:
#salvar dado
partidas_pro.to_csv('bases/partidas_pro.csv', index=False,mode='w') 

**3.1.1 Estatisticas CONTRA**

Tratamento de dados das partidas, para estatísticas.

In [41]:
#Gerar Estatísticas dos confrontos
colunas_numeros = ['pontos_time','gols_marcados','gols_sofridos',
                    'saldo_gols','classificacao','SG_Atq','SG_def',
                    'classificacao_adv']

#colunas para remover
remove_columns = ['pontos_time','gols_marcados','gols_sofridos',
                    'saldo_gols','classificacao','mandante','SG_Atq','SG_def',
                    'classificacao_adv','adv','estadio']

#estatistica
partidas_contra = partidas_geral.copy()

# Ordenar o DataFrame por pessoa e nprova
partidas_contra = partidas_contra.sort_values(by=['adv', 'rodada'])

#for para criar estatisticas
for coluna in colunas_numeros:

    #criar estatiscas para a coluna
    CalculaEstatisticas(partidas_contra, coluna,"adv","partidas_contra")
    print("Estatística criada para coluna",coluna)

#substituir com zeros e remover scouts da rodada
partidas_contra = partidas_contra.drop(remove_columns, axis=1)

#ver
display(partidas_contra.head(5))

Estatística criada para coluna

 pontos_time
Estatística criada para coluna gols_marcados
Estatística criada para coluna gols_sofridos
Estatística criada para coluna saldo_gols
Estatística criada para coluna classificacao
Estatística criada para coluna SG_Atq
Estatística criada para coluna SG_def
Estatística criada para coluna classificacao_adv


,rodada,equipe,partidas_contraAvgpontos_time,partidas_contraLastpontos_time,partidas_contraAvgMov5pontos_time,partidas_contraAvgMov10pontos_time,partidas_contraAvggols_marcados,partidas_contraLastgols_marcados,partidas_contraAvgMov5gols_marcados,partidas_contraAvgMov10gols_marcados,...,partidas_contraAvgMov5SG_Atq,partidas_contraAvgMov10SG_Atq,partidas_contraAvgSG_def,partidas_contraLastSG_def,partidas_contraAvgMov5SG_def,partidas_contraAvgMov10SG_def,partidas_contraAvgclassificacao_adv,partidas_contraLastclassificacao_adv,partidas_contraAvgMov5classificacao_adv,partidas_contraAvgMov10classificacao_adv
371,1,Fluminense,1.0,1.0,1.0,1.0,3.00,3.0,3.00,3.00,...,0.0,0.0,1.00,1.0,1.00,1.00,20.0,20.0,20.0,20.0
361,2,São Paulo,1.0,1.0,1.0,1.0,3.00,3.0,3.00,3.00,...,0.0,0.0,1.00,1.0,1.00,1.00,20.0,20.0,20.0,20.0
353,3,Santos,1.0,1.0,1.0,1.0,3.00,3.0,3.00,3.00,...,0.0,0.0,1.00,1.0,1.00,1.00,20.0,20.0,20.0,20.0
342,4,Cuiabá,1.0,1.0,1.0,1.0,3.00,3.0,3.00,3.00,...,0.0,0.0,0.67,0.0,0.67,0.67,20.0,20.0,20.0,20.0
331,5,Bragantino,1.0,1.0,1.0,1.0,2.75,2.0,2.75,2.75,...,0.0,0.0,0.50,0.0,0.50,0.50,20.0,20.0,20.0,20.0


In [42]:
#salvar dado
partidas_contra.to_csv('bases/partidas_contra.csv', index=False,mode='w') 

**3.2 Jogadores**

Tratamento de dados de pontuações, usandos em clubes e jogadores.

In [43]:
#carregar dados de jogadores
pontos = (pd
.read_csv("C:/Users/eduar/OneDrive/Área de Trabalho/Python/cartola_fc/bronze/bases/cartola_pontos.csv")
.fillna(0)
)

#carregar dados de posicoes
posicoes = (pd
.read_csv("C:/Users/eduar/OneDrive/Área de Trabalho/Python/cartola_fc/bronze/bases/cartola_posicoes.csv")
.drop(['abreviacao'], axis=1)
)

display(pontos.head(5))

,apelido,foto,pontuacao,posicao_id,clube_id,entrou_em_campo,DS,FC,FS,FF,...,GS,FT,CV,PC,V,scout,rodada,DP,PP,GC
0,Zaracho,https://s.sde.globo.com/media/person_role/2023...,3.2,4.0,282.0,True,3.0,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,33,0.0,0.0,0.0
1,Igor Gomes,https://s.sde.globo.com/media/person_role/2023...,2.2,4.0,282.0,True,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,33,0.0,0.0,0.0
2,Pedro Augusto,https://s.sde.globo.com/media/person_role/2023...,1.6,4.0,356.0,True,1.0,3.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,33,0.0,0.0,0.0
3,Yuri Alberto,https://s.sde.globo.com/media/person_role/2023...,8.3,5.0,264.0,True,0.0,1.0,7.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,33,0.0,0.0,0.0
4,Luan Cândido,https://s.sde.globo.com/media/person_role/2022...,2.5,2.0,280.0,True,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,33,0.0,0.0,0.0


In [44]:
display(posicoes)

,id,nome
0,1,Goleiro
1,2,Lateral
2,3,Zagueiro
3,4,Meia
4,5,Atacante
5,6,Técnico


In [45]:
pontos_final = pontos.copy()

#Join, corrige posicao
pontos_final = (pd
.merge(pontos_final, posicoes, left_on='posicao_id', right_on='id', how='left')
.drop(['id'], axis=1)
)

#Join, corrige clubes
pontos_final = (pd
.merge(pontos_final, clubes, left_on='clube_id', right_on='id', how='left')
.drop(['id'], axis=1)
)

#Criar um dicionário de mapeamento de nomes antigos para novos nomes de coluna
novo_nome_colunas = {
    'apelido': 'nome_jogador',
    'nome': 'posicao',
    'nome_fantasia': 'equipe'
}

#Usar a função 'rename()' para renomear as colunas
pontos_final = pontos_final.rename(columns=novo_nome_colunas)

#confrontos
confrontos = partidas_geral[['rodada','equipe','adv']].drop_duplicates()
pontos_final = (pd
.merge(pontos_final, confrontos, left_on=['equipe','rodada'], right_on=['equipe','rodada'], how='left')
)

pontos_final['id_jogador'] = (pontos_final
.apply(lambda row: f"{row['nome_jogador']} - {row['posicao']} ({row['equipe']}){row['foto']}", axis=1)
)

#selecionar dados
pontos_final = (pontos_final[
['rodada','id_jogador','nome_jogador', 'pontuacao','posicao',
'entrou_em_campo', 'equipe',
'adv',
 'DS', 'FC', 'FF',
'FS', 'CA', 'FD', 'FT', 'G', 'DE', 'GS',
'GC', 'I', 'SG', 'A', 'V',
'PS', 'PC', 'CV', 'DP', 'PP']
       ])

#filtra apenas jogadores que jogaram
pontos_final = pontos_final[pontos_final['entrou_em_campo'] == True] #apenas se jogou

#exibir dados
display(pontos_final.head(5))

,rodada,id_jogador,nome_jogador,pontuacao,posicao,entrou_em_campo,equipe,adv,DS,FC,...,GC,I,SG,A,V,PS,PC,CV,DP,PP
0,33,Zaracho - Meia (Atlético-MG)https://s.sde.glob...,Zaracho,3.2,Meia,True,Atlético-MG,Corinthians,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,33,Igor Gomes - Meia (Atlético-MG)https://s.sde.g...,Igor Gomes,2.2,Meia,True,Atlético-MG,Corinthians,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,33,Pedro Augusto - Meia (Fortaleza)https://s.sde....,Pedro Augusto,1.6,Meia,True,Fortaleza,Athlético-PR,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,33,Yuri Alberto - Atacante (Corinthians)https://s...,Yuri Alberto,8.3,Atacante,True,Corinthians,Atlético-MG,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,33,Luan Cândido - Lateral (Bragantino)https://s.s...,Luan Cândido,2.5,Lateral,True,Bragantino,São Paulo,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**3.2.1 Estatística PRO**

Realizar tratamento de dados dos jogadores.

In [46]:
#definir quais colunas serao avaliadas para estatisticas
colunas_numeros = ['pontuacao','DS', 'FC', 'FF',
'FS', 'CA', 'FD', 'FT', 'G', 'DE', 'GS',
 'GC', 'I', 'SG', 'A', 'V',
'PS', 'PC', 'CV', 'DP', 'PP']

#colunas para remover
remove_columns = ['DS', 'FC', 'FF',
'FS', 'CA', 'FD', 'FT', 'G', 'DE', 'GS',
 'GC', 'I', 'SG', 'A', 'V',
'PS', 'PC', 'CV', 'DP', 'PP']

# Ordenar o DataFrame por pessoa e nprova
jogadores = pontos_final.copy().sort_values(by=['id_jogador', 'rodada'])

#for para criar estatisticas
for coluna in colunas_numeros:

    #criar estatiscas para a coluna
    (jogadores, coluna,"id_jogador","jogadores_pro")
    print("Estatística criada para coluna",coluna)

#percentual de jogos que jogou
jogadores['perc_jogos'] = jogadores.groupby('id_jogador').cumcount() / (jogadores['rodada'] -1 )

#replace 1
jogadores['perc_jogos'].fillna(1, inplace=True)

#substituir com zeros e remover scouts da rodada
jogadores = jogadores.drop(remove_columns, axis=1)

Estatística criada para coluna pontuacao
Estatística criada para coluna DS
Estatística criada para coluna FC
Estatística criada para coluna FF
Estatística criada para coluna FS
Estatística criada para coluna CA
Estatística criada para coluna FD
Estatística criada para coluna FT
Estatística criada para coluna G
Estatística criada para coluna DE
Estatística criada para coluna GS
Estatística criada para coluna GC
Estatística criada para coluna I
Estatística criada para coluna SG
Estatística criada para coluna A
Estatística criada para coluna V
Estatística criada para coluna PS
Estatística criada para coluna PC
Estatística criada para coluna CV
Estatística criada para coluna DP
Estatística criada para coluna PP


In [47]:
#print
display(jogadores.head(5))

,rodada,id_jogador,nome_jogador,pontuacao,posicao,entrou_em_campo,equipe,adv,jogadores_proAvgpontuacao,jogadores_proLastpontuacao,...,jogadores_proAvgMov10CV,jogadores_proAvgDP,jogadores_proLastDP,jogadores_proAvgMov5DP,jogadores_proAvgMov10DP,jogadores_proAvgPP,jogadores_proLastPP,jogadores_proAvgMov5PP,jogadores_proAvgMov10PP,perc_jogos
11180,1,Abel Ferreira - Técnico (Palmeiras)https://s.s...,Abel Ferreira,6.25,Técnico,True,Palmeiras,Cuiabá,6.25,6.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10820,2,Abel Ferreira - Técnico (Palmeiras)https://s.s...,Abel Ferreira,5.21,Técnico,True,Palmeiras,Vasco,6.25,6.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10453,3,Abel Ferreira - Técnico (Palmeiras)https://s.s...,Abel Ferreira,5.66,Técnico,True,Palmeiras,Corinthians,5.73,5.21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10101,4,Abel Ferreira - Técnico (Palmeiras)https://s.s...,Abel Ferreira,9.68,Técnico,True,Palmeiras,Goiás,5.71,5.66,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9733,5,Abel Ferreira - Técnico (Palmeiras)https://s.s...,Abel Ferreira,8.58,Técnico,True,Palmeiras,Grêmio,6.70,9.68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [48]:
#salvar dado
jogadores.to_csv('bases/jogadores_pro.csv', index=False,mode='w') 

**3.3 Setor-Equipes**

Realizar tratamento de dados das posicoes clubes.

Devemos fazer um groupby dos times, somandos os scouts dos seus jogadores. E em seguida gerar as estatísticas por rodada.

**3.3.1 Estatística PRO**

Realizar tratamento de dados dos setor-equipes.

In [49]:
#fazer GroupBy para somar scouts por time,setor e por rodada
setor_equipe = pontos_final.copy().groupby(['rodada','posicao','equipe'])[colunas_numeros].sum().reset_index()

#visualizar dados
display(setor_equipe.head(5))

,rodada,posicao,equipe,pontuacao,DS,FC,FF,FS,CA,FD,...,GC,I,SG,A,V,PS,PC,CV,DP,PP
0,1,Atacante,América-MG,11.5,3.0,7.0,6.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Atacante,Athlético-PR,19.5,5.0,6.0,3.0,8.0,1.0,0.0,...,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,Atacante,Atlético-MG,14.9,3.0,3.0,5.0,6.0,2.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Atacante,Bahia,24.6,7.0,6.0,4.0,11.0,1.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Atacante,Botafogo,14.7,4.0,3.0,3.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
#Ordenar o DataFrame
setor_pro = setor_equipe.sort_values(by=[ 'equipe','posicao', 'rodada'])

#for para criar estatisticas
for coluna in colunas_numeros:

    #criar estatiscas para a coluna
    CalculaEstatisticas(setor_pro, coluna,[ 'equipe','posicao'],"setor_pro")
    print("Estatística criada para coluna",coluna)

#colunas para remover
remove_columns = ['DS', 'FC', 'FF',
'FS', 'CA', 'FD', 'FT', 'G', 'DE', 'GS',
 'GC', 'I', 'SG', 'A', 'V',
'PS', 'PC', 'CV', 'DP', 'PP','pontuacao']

#substituir com zeros e remover scouts da rodada
setor_pro = setor_pro.drop(remove_columns, axis=1)

Estatística criada para coluna pontuacao
Estatística criada para coluna DS
Estatística criada para coluna FC
Estatística criada para coluna FF
Estatística criada para coluna FS
Estatística criada para coluna CA
Estatística criada para coluna FD
Estatística criada para coluna FT
Estatística criada para coluna G
Estatística criada para coluna DE
Estatística criada para coluna GS
Estatística criada para coluna GC
Estatística criada para coluna I
Estatística criada para coluna SG
Estatística criada para coluna A
Estatística criada para coluna V
Estatística criada para coluna PS
Estatística criada para coluna PC
Estatística criada para coluna CV
Estatística criada para coluna DP
Estatística criada para coluna PP


In [51]:
#print
display(setor_pro.head(5))

,rodada,posicao,equipe,setor_proAvgpontuacao,setor_proLastpontuacao,setor_proAvgMov5pontuacao,setor_proAvgMov10pontuacao,setor_proAvgDS,setor_proLastDS,setor_proAvgMov5DS,...,setor_proAvgMov5CV,setor_proAvgMov10CV,setor_proAvgDP,setor_proLastDP,setor_proAvgMov5DP,setor_proAvgMov10DP,setor_proAvgPP,setor_proLastPP,setor_proAvgMov5PP,setor_proAvgMov10PP
0,1,Atacante,América-MG,11.50,11.5,11.50,11.50,3.00,3.0,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,2,Atacante,América-MG,11.50,11.5,11.50,11.50,3.00,3.0,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
240,3,Atacante,América-MG,11.85,12.2,11.85,11.85,3.00,3.0,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
360,4,Atacante,América-MG,15.20,21.9,15.20,15.20,3.33,4.0,3.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
480,5,Atacante,América-MG,17.30,23.6,17.30,17.30,3.00,2.0,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
#salvar dado
setor_pro.to_csv('bases/setor_equipes_pro.csv', index=False,mode='w') 

**3.3.1 Estatística CONTRA**

Realizar tratamento de dados dos setor-equipes.

In [53]:
#fazer GroupBy para somar scouts por time,setor e por rodada
setor_equipe = pontos_final.copy().groupby(['rodada','posicao','adv'])[colunas_numeros].sum().reset_index()

#visualizar dados
display(setor_equipe.head(5))

,rodada,posicao,adv,pontuacao,DS,FC,FF,FS,CA,FD,...,GC,I,SG,A,V,PS,PC,CV,DP,PP
0,1,Atacante,América-MG,35.6,3.0,3.0,1.0,4.0,1.0,2.0,...,0.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1,Atacante,Athlético-PR,7.3,2.0,5.0,4.0,4.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,Atacante,Atlético-MG,18.0,4.0,2.0,1.0,2.0,2.0,1.0,...,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Atacante,Bahia,21.6,6.0,7.0,0.0,4.0,0.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Atacante,Botafogo,18.9,1.0,5.0,1.0,8.0,1.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
#Ordenar o DataFrame
setor_contra = setor_equipe.sort_values(by=[ 'adv','posicao', 'rodada'])

#for para criar estatisticas
for coluna in colunas_numeros:

    #criar estatiscas para a coluna
    CalculaEstatisticas(setor_contra, coluna,[ 'adv','posicao'],"partidas_contra")
    print("Estatística criada para coluna",coluna)

#substituir com zeros e remover scouts da rodada
setor_contra = setor_contra.drop(remove_columns, axis=1)

Estatística criada para coluna pontuacao
Estatística criada para coluna DS
Estatística criada para coluna FC
Estatística criada para coluna FF
Estatística criada para coluna FS
Estatística criada para coluna CA
Estatística criada para coluna FD
Estatística criada para coluna FT
Estatística criada para coluna G
Estatística criada para coluna DE
Estatística criada para coluna GS
Estatística criada para coluna GC
Estatística criada para coluna I
Estatística criada para coluna SG
Estatística criada para coluna A
Estatística criada para coluna V
Estatística criada para coluna PS
Estatística criada para coluna PC
Estatística criada para coluna CV
Estatística criada para coluna DP
Estatística criada para coluna PP


In [55]:
#print
display(setor_contra.head(5))

,rodada,posicao,adv,partidas_contraAvgpontuacao,partidas_contraLastpontuacao,partidas_contraAvgMov5pontuacao,partidas_contraAvgMov10pontuacao,partidas_contraAvgDS,partidas_contraLastDS,partidas_contraAvgMov5DS,...,partidas_contraAvgMov5CV,partidas_contraAvgMov10CV,partidas_contraAvgDP,partidas_contraLastDP,partidas_contraAvgMov5DP,partidas_contraAvgMov10DP,partidas_contraAvgPP,partidas_contraLastPP,partidas_contraAvgMov5PP,partidas_contraAvgMov10PP
0,1,Atacante,América-MG,35.60,35.6,35.60,35.60,3.00,3.0,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.00,1.0,1.00,1.00
120,2,Atacante,América-MG,35.60,35.6,35.60,35.60,3.00,3.0,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.00,1.0,1.00,1.00
240,3,Atacante,América-MG,31.50,27.4,31.50,31.50,2.00,1.0,2.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.0,0.50,0.50
360,4,Atacante,América-MG,34.37,40.1,34.37,34.37,3.00,5.0,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.33,0.0,0.33,0.33
480,5,Atacante,América-MG,27.92,8.6,27.92,27.92,3.25,4.0,3.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.25,0.25


In [56]:
#salvar dado
setor_contra.to_csv('bases/setor_equipes_contra.csv', index=False,mode='w') 

**3.4 Equipes**

Realizar tratamento de dados das posicoes clubes.

Devemos fazer um groupby dos times, somandos os scouts dos seus jogadores. E em seguida gerar as estatísticas por rodada.

**3.4.1 Estatística PRO**

Realizar tratamento de dados de equipes.

In [57]:
#fazer GroupBy para somar scouts por time e por rodada
equipe = pontos_final.copy().groupby(['rodada','equipe'])[colunas_numeros].sum().reset_index()

#visualizar dados
display(equipe.head(5))

,rodada,equipe,pontuacao,DS,FC,FF,FS,CA,FD,FT,...,GC,I,SG,A,V,PS,PC,CV,DP,PP
0,1,América-MG,32.40,11.0,21.0,10.0,7.0,2.0,2.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,Athlético-PR,79.87,11.0,18.0,4.0,16.0,2.0,2.0,0.0,...,0.0,1.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,Atlético-MG,53.45,20.0,15.0,9.0,9.0,6.0,10.0,0.0,...,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,Bahia,57.16,20.0,13.0,5.0,20.0,1.0,3.0,0.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Botafogo,62.42,20.0,20.0,6.0,9.0,3.0,1.0,0.0,...,0.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0


In [58]:
#Ordenar o DataFrame
equipe_pro = equipe.sort_values(by=[ 'equipe', 'rodada'])

#for para criar estatisticas
for coluna in colunas_numeros:

    #criar estatiscas para a coluna
    CalculaEstatisticas(equipe_pro, coluna,[ 'equipe'],"equipe_pro")
    print("Estatística criada para coluna",coluna)

#substituir com zeros e remover scouts da rodada
equipe_pro = equipe_pro.drop(remove_columns, axis=1)

Estatística criada para coluna pontuacao
Estatística criada para coluna DS
Estatística criada para coluna FC
Estatística criada para coluna FF
Estatística criada para coluna FS
Estatística criada para coluna CA
Estatística criada para coluna FD
Estatística criada para coluna FT
Estatística criada para coluna G
Estatística criada para coluna DE
Estatística criada para coluna GS
Estatística criada para coluna GC
Estatística criada para coluna I
Estatística criada para coluna SG
Estatística criada para coluna A
Estatística criada para coluna V
Estatística criada para coluna PS
Estatística criada para coluna PC
Estatística criada para coluna CV
Estatística criada para coluna DP
Estatística criada para coluna PP


In [59]:
#print
display(equipe_pro.head(5))

,rodada,equipe,equipe_proAvgpontuacao,equipe_proLastpontuacao,equipe_proAvgMov5pontuacao,equipe_proAvgMov10pontuacao,equipe_proAvgDS,equipe_proLastDS,equipe_proAvgMov5DS,equipe_proAvgMov10DS,...,equipe_proAvgMov5CV,equipe_proAvgMov10CV,equipe_proAvgDP,equipe_proLastDP,equipe_proAvgMov5DP,equipe_proAvgMov10DP,equipe_proAvgPP,equipe_proLastPP,equipe_proAvgMov5PP,equipe_proAvgMov10PP
0,1,América-MG,32.40,32.40,32.40,32.40,11.00,11.0,11.00,11.00,...,0.00,0.00,1.00,1.0,1.00,1.00,0.0,0.0,0.0,0.0
20,2,América-MG,32.40,32.40,32.40,32.40,11.00,11.0,11.00,11.00,...,0.00,0.00,1.00,1.0,1.00,1.00,0.0,0.0,0.0,0.0
40,3,América-MG,31.42,30.44,31.42,31.42,10.50,10.0,10.50,10.50,...,0.00,0.00,0.50,0.0,0.50,0.50,0.0,0.0,0.0,0.0
60,4,América-MG,41.67,62.18,41.67,41.67,11.67,14.0,11.67,11.67,...,0.00,0.00,0.33,0.0,0.33,0.33,0.0,0.0,0.0,0.0
80,5,América-MG,39.68,33.71,39.68,39.68,11.25,10.0,11.25,11.25,...,0.25,0.25,0.25,0.0,0.25,0.25,0.0,0.0,0.0,0.0


In [60]:
#salvar dado
equipe_pro.to_csv('bases/equipes_pro.csv', index=False,mode='w') 

**3.4.2 Estatística CONTRA**

Realizar tratamento de dados de equipes.

In [61]:
#fazer GroupBy para somar scouts por adv e por rodada
equipe = pontos_final.copy().groupby(['rodada','adv'])[colunas_numeros].sum().reset_index()

#visualizar dados
display(equipe.head(5))

,rodada,adv,pontuacao,DS,FC,FF,FS,CA,FD,FT,...,GC,I,SG,A,V,PS,PC,CV,DP,PP
0,1,América-MG,113.69,12.0,9.0,2.0,23.0,2.0,4.0,1.0,...,0.0,3.0,7.0,3.0,1.0,1.0,0.0,0.0,0.0,1.0
1,1,Athlético-PR,18.87,6.0,18.0,9.0,18.0,3.0,2.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,Atlético-MG,61.55,16.0,10.0,1.0,15.0,5.0,1.0,0.0,...,0.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1,Bahia,68.75,29.0,20.0,4.0,13.0,3.0,4.0,1.0,...,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,Botafogo,54.87,19.0,12.0,5.0,20.0,2.0,6.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
#Ordenar o DataFrame
equipe_contra = equipe.sort_values(by=[ 'adv', 'rodada'])

#for para criar estatisticas
for coluna in colunas_numeros:

    #criar estatiscas para a coluna
    CalculaEstatisticas(equipe_contra, coluna,[ 'adv'],"equipe_contra")
    print("Estatística criada para coluna",coluna)

#substituir com zeros e remover scouts da rodada
equipe_contra = equipe_contra.drop(remove_columns, axis=1)

Estatística criada para coluna pontuacao
Estatística criada para coluna DS
Estatística criada para coluna FC
Estatística criada para coluna FF
Estatística criada para coluna FS
Estatística criada para coluna CA
Estatística criada para coluna FD
Estatística criada para coluna FT
Estatística criada para coluna G
Estatística criada para coluna DE
Estatística criada para coluna GS
Estatística criada para coluna GC
Estatística criada para coluna I
Estatística criada para coluna SG
Estatística criada para coluna A
Estatística criada para coluna V
Estatística criada para coluna PS
Estatística criada para coluna PC
Estatística criada para coluna CV
Estatística criada para coluna DP
Estatística criada para coluna PP


In [63]:
#print
display(equipe_contra.head(5))

,rodada,adv,equipe_contraAvgpontuacao,equipe_contraLastpontuacao,equipe_contraAvgMov5pontuacao,equipe_contraAvgMov10pontuacao,equipe_contraAvgDS,equipe_contraLastDS,equipe_contraAvgMov5DS,equipe_contraAvgMov10DS,...,equipe_contraAvgMov5CV,equipe_contraAvgMov10CV,equipe_contraAvgDP,equipe_contraLastDP,equipe_contraAvgMov5DP,equipe_contraAvgMov10DP,equipe_contraAvgPP,equipe_contraLastPP,equipe_contraAvgMov5PP,equipe_contraAvgMov10PP
0,1,América-MG,113.69,113.69,113.69,113.69,12.00,12.0,12.00,12.00,...,0.00,0.00,0.0,0.0,0.0,0.0,1.00,1.0,1.00,1.00
20,2,América-MG,113.69,113.69,113.69,113.69,12.00,12.0,12.00,12.00,...,0.00,0.00,0.0,0.0,0.0,0.0,1.00,1.0,1.00,1.00
40,3,América-MG,108.02,102.35,108.02,108.02,12.50,13.0,12.50,12.50,...,0.00,0.00,0.0,0.0,0.0,0.0,0.50,0.0,0.50,0.50
60,4,América-MG,94.49,67.44,94.49,94.49,14.33,18.0,14.33,14.33,...,0.33,0.33,0.0,0.0,0.0,0.0,0.33,0.0,0.33,0.33
80,5,América-MG,82.46,46.38,82.46,82.46,15.75,20.0,15.75,15.75,...,0.25,0.25,0.0,0.0,0.0,0.0,0.25,0.0,0.25,0.25


In [64]:
#salvar dado
equipe_contra.to_csv('bases/equipes_contra.csv', index=False,mode='w')